In [20]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import keras
import os
import pandas as pd  
import tensorflow_datasets as tfds

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

## 用xception 预训练 tf_flowers数据集

In [6]:

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
print(info.splits)
#只有"train"训练集，没有测试集和验证集，所以需要分割训练集。

dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names 
n_classes = info.features["label"].num_classes

{'train': <tfds.core.SplitInfo num_examples=3670>}


In [3]:
train_set,valid_set,test_set = tfds.load("tf_flowers", 
                                         split=["train[:75%]", "train[75%:90%]", "train[:10%]"],
                                         as_supervised=True)
print("train_set:",len(list(train_set)))
print("valid_set:",len(list(valid_set)))
print("test_set:",len(list(test_set)))

train_set: 2752
valid_set: 551
test_set: 367


In [4]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1) 

#如果想做数据增强，可以修改训练集的预处理函数，给训练图片添加一些转换。
#使用tf.image.random_crop()随机裁剪图片，
#使用tf.image.random_flip_left_right()做随机水平翻转

In [11]:
#加载一个在 ImageNet 上预训练的 Xception 模型。
#通过设定include_top=False，排除模型的顶层,默认模型接受299*299图片，删除重新添加input适配 none*none
#排除了全局平均池化层和紧密输出层。我们然后根据基本模型的输出
#添加自己的全局平均池化层，然后添加紧密输出层，默认1000类，现在五类。
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [18]:
model.input

<KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input_3')>

In [17]:
#冻结模型参数，因为我们的模型直接使用了基本模型的层，而不是base_model对象
#设置base_model.trainable=False没有任何效果。
for layer in base_model.layers:
    layer.trainable = False